In [1]:
!nvidia-smi

Sun Apr 23 02:52:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   35C    P0    46W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # mount driver
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CardiacSeg/eda/Network
#
# #install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# !pip install ray
# %matplotlib inline

In [3]:
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.networkx.blocks.ham import HAM
from networks.ssl_head import SSLHead

# sync python module
%load_ext autoreload
%autoreload 2

In [7]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 2
args.drop_rate = 0.4
args.depths = [3, 3, 9, 3]
args.use_init_weights = True
net = None
del net
net = network('unetcnx_a0', args) # unetc2f
inp = None
del inp
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

model: unetcnx_a0
patch size: 2
drop rate: 0.4
depths: [3, 3, 9, 3]
True
use init w
                                                          Kernel Shape  \
Layer                                                                    
0_encoder0.layer.conv1.Conv3d_conv                    [1, 24, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                                -   
2_encoder0.layer.LeakyReLU_lrelu                                     -   
3_encoder0.layer.conv2.Conv3d_conv                   [24, 24, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                                -   
5_encoder0.layer.conv3.Conv3d_conv                    [1, 24, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                                -   
7_encoder0.layer.LeakyReLU_lrelu                                     -   
8_stem.Conv3d_0                                       [1, 48, 2, 2, 2]   
9_stem.Conv3d_0                                       [1, 48, 2, 2, 2]   
10_stem.LayerNorm_1         

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder0.layer.conv1.Conv3d_conv,"[1, 24, 3, 3, 3]","[1, 24, 96, 96, 96]",648.0,5.733089e+08
1_encoder0.layer.InstanceNorm3d_norm1,-,"[1, 24, 96, 96, 96]",NaN,NaN
2_encoder0.layer.LeakyReLU_lrelu,-,"[1, 24, 96, 96, 96]",NaN,NaN
3_encoder0.layer.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 96, 96, 96]",15552.0,1.375941e+10
4_encoder0.layer.InstanceNorm3d_norm2,-,"[1, 24, 96, 96, 96]",NaN,NaN
...,...,...,...,...
357_decoder1.conv_block.InstanceNorm3d_norm2,-,"[1, 24, 96, 96, 96]",NaN,NaN
358_decoder1.conv_block.conv3.Conv3d_conv,"[48, 24, 1, 1, 1]","[1, 24, 96, 96, 96]",1152.0,1.019216e+09
359_decoder1.conv_block.InstanceNorm3d_norm3,-,"[1, 24, 96, 96, 96]",NaN,NaN


In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))